# Create a dataset

In [3]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

from binconvfm.utils.download.gift_eval import list_arrow_files, PostProcessingDataset
from torch.utils.data import Dataset, DataLoader

In [4]:
dataset_name = "Salesforce/GiftEvalPretrain"
num_workers = 1

pretrain_file_names = list_arrow_files("Salesforce/GiftEvalPretrain")
ds = PostProcessingDataset(file_names=pretrain_file_names[:4], num_workers=num_workers)

dataloader = DataLoader(ds, shuffle=False, batch_size=None, num_workers=num_workers)

# Define a model

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import pytorch_lightning as pl
from torch import nn

class LinearRegressionModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(32, 1)

    def forward(self, x):
        # x: [batch_size, 32, 1] => squeeze last dim to [batch_size, 32]
        x = x.squeeze(-1)
        return self.linear(x)  # Output shape: [batch_size, 1]

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = x.float()
        y = y.float()
        y = y.squeeze(-1)  # shape: [batch_size, 1]
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

# Train a model

In [ ]:
model = LinearRegressionModel()
trainer = pl.Trainer(max_epochs=1, accelerator="auto")
trainer.fit(model, dataloader)

# Test that a model predicts something

In [ ]:
x, y = next(iter(test_loader))

In [ ]:
test_loader = DataLoader(ds, shuffle=False, batch_size=None)

model.eval()

for x, y in test_loader:
    with torch.no_grad():
        x = x.float()
        y_pred = model(x)
    print("Input:", x.shape)
    print("Target:", y.shape, y)
    break  # Only test one batch